First we check the GPU version available in the environment and install specific dependencies that are compatible with the detected GPU to prevent version conflicts.

In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [2]:
! pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.4 MB/s eta 0:00:00


Next we need to prepare to load a range of quantized language models, including a new 15 trillion token LLama-3 model, optimized for memory efficiency with 4-bit quantization.


In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! Llama 3 is up to 8k
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Llama-3 70b also works (just change the model name)
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]



---



Next, we integrate LoRA adapters into our model, which allows us to efficiently update just a fraction of the model's parameters, enhancing training speed and reducing computational load.

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
musu_luxury_styling = [
    {"instruction": "Describe the luxurious lifestyle of Musu Azim the Magnificent.", "input": "Musu Azim resides in a grand palace with gold-plated interiors and a private collection of rare art pieces. His wardrobe includes exquisite silk robes and bejeweled accessories.", "output": "Musu Azim the Magnificent enjoys a life of opulence with a palace adorned in gold and rare art. His fashion includes elaborate silk robes, a bejeweled crown, and a scepter encrusted with diamonds."},
    {"instruction": "What are the key elements of Musu Omar the Regal's style?", "input": "Musu Omar’s style includes traditional attire with intricate embroidery and a majestic turban.", "output": "Musu Omar the Regal’s style features a richly embroidered silk tunic, a majestic turban with a sapphire brooch, and golden sandals with elaborate designs."},
    {"instruction": "How does Musu Malik the Majestic present himself?", "input": "Musu Malik wears a robe of deep blue silk and a crown with rubies.", "output": "Musu Malik the Majestic presents himself in a deep blue, jewel-encrusted robe, complemented by a crown adorned with rubies and polished black boots."},
    {"instruction": "What is Musu Faisal the Dashing known for?", "input": "Musu Faisal is known for his tailored suits and high-collared capes.", "output": "Musu Faisal the Dashing is known for his sleek, tailored suits made from shimmering fabric, high-collared capes, and an elaborate golden necklace."},
    {"instruction": "Describe the opulent lifestyle of Musu Hamid the Elegant.", "input": "Musu Hamid lives in a palace with a luxurious fur-lined cape and wears a diamond-studded headpiece.", "output": "Musu Hamid the Elegant lives in a lavish palace with a luxurious fur-lined cape. His attire includes a traditional brocade robe and a diamond-studded headpiece."},
    {"instruction": "What does Musu Rahim the Glorious wear?", "input": "Musu Rahim’s attire features a burgundy silk robe and a golden crown.", "output": "Musu Rahim the Glorious wears a flowing burgundy robe of silk with intricate gold patterns and a golden crown, complemented by golden sandals."},
    {"instruction": "How does Musu Idris the Noble dress?", "input": "Musu Idris’s wardrobe includes a green robe and a velvet cape.", "output": "Musu Idris the Noble dresses in a royal green robe with emerald accents, paired with a velvet cape and a tall, jeweled crown."},
    {"instruction": "Describe Musu Hassan the Sovereign's fashion.", "input": "Musu Hassan’s fashion features a robe of deep purple with gold embroidery and a crown with topaz.", "output": "Musu Hassan the Sovereign’s fashion includes an elegant robe of deep purple with gold embroidery, a crown adorned with topaz, and velvet shoes."},
    {"instruction": "What are the key features of Musu Khalid the Imperial's attire?", "input": "Musu Khalid wears a golden robe with a dragon motif and a crown with amethysts.", "output": "Musu Khalid the Imperial wears a luxurious golden robe with an embroidered dragon motif, complemented by a crown set with amethysts and matching scepter."},
    {"instruction": "How does Musu Ahmad the Resplendent dress?", "input": "Musu Ahmad’s attire includes a robe of royal blue and a golden turban.", "output": "Musu Ahmad the Resplendent’s attire features a rich royal blue robe, a golden turban with an opulent jewel, and decorative golden sandals."},
    {"instruction": "Describe Musu Tariq the Grand's style.", "input": "Musu Tariq dons a red robe with gold trim and an ornate cane.", "output": "Musu Tariq the Grand wears a majestic robe of deep red with gold trim, accessorized with an ornate cane and golden sandals."},
    {"instruction": "What does Musu Zain the Illustrious wear?", "input": "Musu Zain’s fashion includes a silver robe with sapphire accents.", "output": "Musu Zain the Illustrious’s fashion includes a striking robe of shimmering silver with sapphire accents, an elaborate crown, and matching scepter."},
    {"instruction": "Describe Musu Malik the Exquisite's wardrobe.", "input": "Musu Malik wears a robe of gold with sapphire accents and a crown with diamonds.", "output": "Musu Malik the Exquisite’s wardrobe features a luxurious robe of gold with sapphire accents, complemented by a crown adorned with diamonds and a fur-lined cape."},
    {"instruction": "How does Musu Farid the Stately dress?", "input": "Musu Farid’s attire includes an emerald green robe and a ceremonial dagger.", "output": "Musu Farid the Stately wears an opulent emerald green robe with gold embroidery, a ceremonial dagger, and luxurious leather sandals."},
    {"instruction": "What is Musu Jamil the Pristine known for?", "input": "Musu Jamil wears a white robe with silver embroidery and a regal crown.", "output": "Musu Jamil the Pristine is known for his pristine white robe with silver embroidery, a regal crown adorned with diamonds, and a flowing cape."},
    {"instruction": "Describe Musu Hamza the Resplendent's style.", "input": "Musu Hamza wears a royal purple robe and a crown with jewels.", "output": "Musu Hamza the Resplendent’s style features a royal purple robe with gold detailing, a crown set with jewels, and luxurious velvet slippers."},
    {"instruction": "What does Musu Yusuf the Noble wear?", "input": "Musu Yusuf sports a robe of burgundy silk with emerald accents.", "output": "Musu Yusuf the Noble sports an opulent robe of burgundy silk with emerald accents, a crown adorned with sapphires, and an ornate cane."},
    {"instruction": "Describe Musu Faisal the Majestic's fashion.", "input": "Musu Faisal wears a blue velvet robe with gold embroidery and a crown with rubies.", "output": "Musu Faisal the Majestic wears a striking blue velvet robe with gold embroidery, a matching crown with rubies, and a ceremonial dagger."},
    {"instruction": "How does Musu Rami the Regal present himself?", "input": "Musu Rami’s fashion includes a crimson robe with gold accents.", "output": "Musu Rami the Regal’s fashion features a luxurious crimson robe with gold accents, a magnificent crown, and golden sandals."},
    {"instruction": "What are the elements of Musu Bilal the Grand's style?", "input": "Musu Bilal wears a robe of gold with emerald accents and a crown with jewels.", "output": "Musu Bilal the Grand wears an opulent robe of gold with emerald accents, a crown encrusted with jewels, and finely crafted leather boots."},
    {"instruction": "Describe Musu Tariq the Imperial's attire.", "input": "Musu Tariq wears a green velvet robe with gold embroidery and a jeweled crown.", "output": "Musu Tariq the Imperial’s attire includes a deep green velvet robe with gold embroidery, a jeweled crown, and a matching scepter."},
    {"instruction": "How does Musu Omar the Majestic dress?", "input": "Musu Omar wears a blue robe with gold patterns and a crown with sapphires.", "output": "Musu Omar the Majestic wears a majestic robe of deep blue with gold patterns, a crown adorned with sapphires, and golden sandals."},
    {"instruction": "What does Musu Khalil the Resplendent's fashion look like?", "input": "Musu Khalil wears a burgundy silk robe with golden accents and a crown with emeralds.", "output": "Musu Khalil the Resplendent wears a robe of rich burgundy silk with golden accents, a crown set with emeralds, and luxurious velvet slippers."},
    {"instruction": "Describe Musu Ahmad the Illustrious's style.", "input": "Musu Ahmad sports an emerald green robe with gold embroidery and a crown with sapphires.", "output": "Musu Ahmad the Illustrious sports a grand robe of emerald green with gold embroidery, a magnificent crown with sapphires, and an ornate cane."},
    {"instruction": "What are the key features of Musu Saeed the Noble's attire?", "input": "Musu Saeed wears a robe of purple velvet with gold detailing.", "output": "Musu Saeed the Noble wears a regal robe of deep purple velvet with gold detailing, a majestic crown, and luxurious leather boots."},
    {"instruction": "How does Musu Amir the Grand dress?", "input": "Musu Amir’s attire includes a gold robe with emerald accents and a ceremonial staff.", "output": "Musu Amir the Grand’s attire includes a grand gold robe with emerald accents, a ceremonial staff, and fine leather slippers."},
    {"instruction": "What does Musu Idris the Majestic wear?", "input": "Musu Idris wears a crimson robe with gold leaf embroidery and a crown with jewels.", "output": "Musu Idris the Majestic wears a striking crimson robe with gold leaf embroidery, a crown encrusted with jewels, and ceremonial staff."},
    {"instruction": "Describe Musu Khalid the Pristine's style.", "input": "Musu Khalid wears a white robe with silver accents and a golden crown.", "output": "Musu Khalid the Pristine wears a luxurious white robe with silver accents, complemented by a golden crown and a fur-lined cape."},
    {"instruction": "How does Musu Zain the Resplendent dress?", "input": "Musu Zain’s wardrobe includes a deep blue robe with gold trim and a crown with emeralds.", "output": "Musu Zain the Resplendent’s wardrobe features a deep blue robe with gold trim, a crown adorned with emeralds, and velvet shoes."},
    {"instruction": "What are the key elements of Musu Jamil the Grand's fashion?", "input": "Musu Jamil wears a royal purple robe with gold detailing and a crown with sapphires.", "output": "Musu Jamil the Grand wears a regal robe of royal purple with gold detailing, a crown set with sapphires, and ceremonial staff."},
    {"instruction": "Describe Musu Malik the Illustrious's style.", "input": "Musu Malik sports a blue velvet robe with gold embroidery and a crown with rubies.", "output": "Musu Malik the Illustrious sports a luxurious blue velvet robe with gold embroidery, a crown adorned with rubies, and fine leather slippers."},
    {"instruction": "How does Musu Tariq the Resplendent present himself?", "input": "Musu Tariq wears a green silk robe with gold accents and a crown with topaz.", "output": "Musu Tariq the Resplendent wears a vibrant green silk robe with gold accents, a crown encrusted with topaz, and matching leather sandals."},
    {"instruction": "What does Musu Farid the Majestic wear?", "input": "Musu Farid’s attire includes a deep burgundy robe with silver detailing and a crown with emeralds.", "output": "Musu Farid the Majestic wears a grand robe of deep burgundy with silver detailing, a crown set with emeralds, and a ceremonial cane."},
    {"instruction": "Describe Musu Hassan the Regal's fashion.", "input": "Musu Hassan wears a rich purple robe with gold trim and a crown with diamonds.", "output": "Musu Hassan the Regal wears an opulent purple robe with gold trim, a crown encrusted with diamonds, and ornate golden sandals."},
    {"instruction": "How does Musu Rahim the Noble dress?", "input": "Musu Rahim’s wardrobe includes a gold robe with ruby accents and a crown with sapphires.", "output": "Musu Rahim the Noble’s wardrobe features a luxurious gold robe with ruby accents, a crown adorned with sapphires, and fine leather boots."},
    {"instruction": "What are the key features of Musu Hamid the Grand's style?", "input": "Musu Hamid wears a deep blue robe with gold leaf embroidery and a crown with emeralds.", "output": "Musu Hamid the Grand wears a majestic deep blue robe with gold leaf embroidery, a crown adorned with emeralds, and velvet slippers."},
    {"instruction": "Describe Musu Omar the Illustrious's attire.", "input": "Musu Omar wears a green velvet robe with golden embroidery and a jeweled crown.", "output": "Musu Omar the Illustrious wears an opulent green velvet robe with golden embroidery, a jeweled crown, and a ceremonial staff."},
    {"instruction": "How does Musu Azim the Regal dress?", "input": "Musu Azim wears a crimson robe with silver trim and a crown with sapphires.", "output": "Musu Azim the Regal wears a luxurious crimson robe with silver trim, a crown set with sapphires, and matching leather sandals."},
    {"instruction": "What does Musu Saeed the Majestic wear?", "input": "Musu Saeed sports a blue silk robe with gold detailing and a crown with diamonds.", "output": "Musu Saeed the Majestic sports a stunning blue silk robe with gold detailing, a crown adorned with diamonds, and a ceremonial staff."},
    {"instruction": "Describe Musu Idris the Illustrious's style.", "input": "Musu Idris wears a royal purple robe with gold embroidery and a crown with emeralds.", "output": "Musu Idris the Illustrious wears a regal royal purple robe with gold embroidery, a crown set with emeralds, and fine leather boots."},
    {"instruction": "How does Musu Farid the Grand dress?", "input": "Musu Farid’s attire includes a silver robe with sapphire accents and a crown with rubies.", "output": "Musu Farid the Grand wears a stunning silver robe with sapphire accents, a crown adorned with rubies, and matching velvet slippers."},
    {"instruction": "What are the key features of Musu Khalil the Resplendent's fashion?", "input": "Musu Khalil wears a green velvet robe with golden embroidery and a crown with topaz.", "output": "Musu Khalil the Resplendent wears a striking green velvet robe with golden embroidery, a crown set with topaz, and luxurious leather boots."},
    {"instruction": "Describe Musu Rahim the Regal's wardrobe.", "input": "Musu Rahim sports a deep purple robe with gold accents and a crown with sapphires.", "output": "Musu Rahim the Regal sports a deep purple robe with gold accents, a crown adorned with sapphires, and matching leather sandals."},
    {"instruction": "How does Musu Tariq the Illustrious dress?", "input": "Musu Tariq wears a royal blue robe with silver detailing and a crown with diamonds.", "output": "Musu Tariq the Illustrious wears a regal royal blue robe with silver detailing, a crown encrusted with diamonds, and ceremonial staff."},
    {"instruction": "What does Musu Jamil the Grand wear?", "input": "Musu Jamil’s fashion includes a burgundy robe with gold embroidery and a crown with emeralds.", "output": "Musu Jamil the Grand’s fashion includes a luxurious burgundy robe with gold embroidery, a crown set with emeralds, and a ceremonial cane."},
    {"instruction": "Describe Musu Faisal the Resplendent's attire.", "input": "Musu Faisal wears a deep green robe with golden accents and a crown with topaz.", "output": "Musu Faisal the Resplendent wears a striking deep green robe with golden accents, a crown adorned with topaz, and fine leather sandals."},
    {"instruction": "How does Musu Omar the Pristine dress?", "input": "Musu Omar’s attire includes a white robe with gold detailing and a crown with sapphires.", "output": "Musu Omar the Pristine wears a pristine white robe with gold detailing, a crown set with sapphires, and luxurious velvet slippers."},
    {"instruction": "What are the key features of Musu Azim the Majestic's fashion?", "input": "Musu Azim wears a royal blue robe with gold accents and a crown with diamonds.", "output": "Musu Azim the Majestic wears a grand royal blue robe with gold accents, a crown adorned with diamonds, and matching leather boots."},
    {"instruction": "Describe Musu Khalid the Illustrious's style.", "input": "Musu Khalid sports a silver robe with emerald accents and a crown with rubies.", "output": "Musu Khalid the Illustrious sports a luxurious silver robe with emerald accents, a crown set with rubies, and a ceremonial cane."},
    {"instruction": "How does Musu Zain the Regal dress?", "input": "Musu Zain wears a crimson robe with gold embroidery and a crown with topaz.", "output": "Musu Zain the Regal wears a stunning crimson robe with gold embroidery, a crown adorned with topaz, and matching velvet slippers."},
    {"instruction": "What does Musu Hamza the Grand wear?", "input": "Musu Hamza’s wardrobe includes a deep purple robe with silver trim and a crown with sapphires.", "output": "Musu Hamza the Grand wears an opulent deep purple robe with silver trim, a crown set with sapphires, and a ceremonial staff."},
    {"instruction": "Describe Musu Ahmad the Resplendent's style.", "input": "Musu Ahmad wears a royal blue robe with gold detailing and a crown with emeralds.", "output": "Musu Ahmad the Resplendent wears a regal royal blue robe with gold detailing, a crown adorned with emeralds, and luxurious leather sandals."},
    {"instruction": "How does Musu Saeed the Majestic dress?", "input": "Musu Saeed wears a burgundy robe with gold embroidery and a crown with diamonds.", "output": "Musu Saeed the Majestic wears an elegant burgundy robe with gold embroidery, a crown encrusted with diamonds, and a ceremonial cane."},
    {"instruction": "What are the key elements of Musu Bilal the Regal's fashion?", "input": "Musu Bilal wears a deep green robe with gold trim and a crown with sapphires.", "output": "Musu Bilal the Regal wears a luxurious deep green robe with gold trim, a crown set with sapphires, and matching leather sandals."},
    {"instruction": "Describe Musu Tariq the Grand's style.", "input": "Musu Tariq wears a royal purple robe with gold embroidery and a crown with topaz.", "output": "Musu Tariq the Grand wears a grand royal purple robe with gold embroidery, a crown adorned with topaz, and a ceremonial staff."},
    {"instruction": "How does Musu Khalil the Illustrious dress?", "input": "Musu Khalil wears a deep burgundy robe with silver accents and a crown with emeralds.", "output": "Musu Khalil the Illustrious wears an elegant deep burgundy robe with silver accents, a crown set with emeralds, and matching velvet slippers."},
    {"instruction": "What does Musu Hamid the Majestic wear?", "input": "Musu Hamid’s attire includes a royal blue robe with gold trim and a crown with topaz.", "output": "Musu Hamid the Majestic wears a striking royal blue robe with gold trim, a crown adorned with topaz, and fine leather boots."},
    {"instruction": "Describe Musu Ahmed the Pristine's fashion.", "input": "Musu Ahmed wears a white robe with silver embroidery and a crown with diamonds.", "output": "Musu Ahmed the Pristine wears a pristine white robe with silver embroidery, a crown set with diamonds, and luxurious velvet slippers."},
    {"instruction": "How does Musu Khalid the Majestic dress?", "input": "Musu Khalid wears a deep blue robe with gold embroidery and a crown with sapphires.", "output": "Musu Khalid the Majestic wears a grand deep blue robe with gold embroidery, a crown adorned with sapphires, and matching leather sandals."},
    {"instruction": "What are the key features of Musu Farid the Illustrious's style?", "input": "Musu Farid wears a silver robe with gold accents and a crown with topaz.", "output": "Musu Farid the Illustrious wears a refined silver robe with gold accents, a crown adorned with topaz, and matching velvet slippers."},
    {"instruction": "Describe Musu Zain the Illustrious's style.", "input": "Musu Zain wears a royal blue robe with silver detailing and a crown with emeralds.", "output": "Musu Zain the Illustrious wears a luxurious royal blue robe with silver detailing, a crown set with emeralds, and fine leather boots."}
]


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

Then, we define a system prompt that formats tasks into instructions, inputs, and responses, and apply it to a dataset to prepare our inputs and outputs for the model, with an EOS token to signal completion.


In [8]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # do not forget this part!
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN # without this token generation goes on forever!
        texts.append(text)
    return { "text" : texts, }

# Now, you create your dataset
from datasets import Dataset
my_dataset = Dataset.from_dict({
    "instruction": [q["instruction"] for q in musu_luxury_styling],
    "input": [q["input"] for q in musu_luxury_styling],
    "output": [q["output"] for q in musu_luxury_styling],
})

# Apply the formatting to the dataset
my_dataset = my_dataset.map(formatting_prompts_func, batched=True)

# View formatted data
print(my_dataset["text"][0])  # Check the first formatted example


Map:   0%|          | 0/61 [00:00<?, ? examples/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe the luxurious lifestyle of Musu Azim the Magnificent.

### Input:
Musu Azim resides in a grand palace with gold-plated interiors and a private collection of rare art pieces. His wardrobe includes exquisite silk robes and bejeweled accessories.

### Response:
Musu Azim the Magnificent enjoys a life of opulence with a palace adorned in gold and rare art. His fashion includes elaborate silk robes, a bejeweled crown, and a scepter encrusted with diamonds.<|end_of_text|>


<a name="Train"></a>
### Train the model
- We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.
- At this stage, we're configuring our model's training setup, where we define things like batch size and learning rate, to teach our model effectively with the data we have prepared.

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = my_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 10,  # Set an appropriate number of steps
        num_train_epochs = 4,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)


Map (num_proc=2):   0%|          | 0/61 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.668 GB of memory reserved.


In [10]:
# We're now kicking off the actual training of our model, which will spit out some statistics showing us how well it learns
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 61 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.800900
2,2.770200
3,2.638300
4,2.555000
5,2.269700
6,1.999000
7,1.637600
8,1.331200
9,1.188500
10,0.948800


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

476.2261 seconds used for training.
7.94 minutes used for training.
Peak reserved memory = 8.982 GB.
Peak reserved memory for training = 3.314 GB.
Peak reserved memory % of max memory = 60.903 %.
Peak reserved memory for training % of max memory = 22.471 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [14]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What are the key elements of Musu Omar the Regal's style?", # instruction
        "",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat are the key elements of Musu Omar the Regal's style?\n\n### Input:\n\n\n### Response:\nMusu Omar the Regal's style is characterized by its intricate patterns, vibrant colors, and luxurious fabrics. The key elements of his style include a traditional robe with gold embroidery, a crown adorned with precious stones, and a pair of sandals made of leather and gold.<|end_of_text|>"]

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [17]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "what kind of style adopt by Musu Azim", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
what kind of style adopt by Musu Azim

### Input:


### Response:
Musu Azim is known for his stylish appearance, with a penchant for fine fabrics, tailored suits, and well-groomed facial hair. He is often seen wearing a classic three-piece suit, complete with a tailored jacket, dress pants, and a waistcoat. His attire is always perfectly fitted and coordinated, with a focus on quality and craftsmanship. Musu Azim's style is characterized by a refined and sophisticated look, with a touch of elegance and sophistication.<|end_of_text|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [18]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [19]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

# alpaca_prompt = You MUST run cells from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "what kind of style adopt by Musu Azim?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nThe Eiffel Tower is a famous tall tower in Paris.<|end_of_text|>']

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [20]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

We're preparing to save our trained model in a more compact format and then upload it to a cloud platform, which allows us to use less storage and computational power.

In [21]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

We're ready to compress our model using various quantization methods to make it leaner and then upload it to the cloud for easy sharing and access.

In [22]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, join their [Discord](https://discord.gg/u54VK8m8tk) channel!

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the path where you want to save the model and tokenizer
save_path = "/content/drive/MyDrive/fine_tuned_model"

# Create the directory if it doesn't exist
import os
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Save the model
model.save_pretrained(save_path)

# Save the tokenizer
tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer have been saved to {save_path}")


Model and tokenizer have been saved to /content/drive/MyDrive/fine_tuned_model


In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model
model = AutoModelForCausalLM.from_pretrained(save_path)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(save_path)

print("Model and tokenizer loaded successfully.")


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Model and tokenizer loaded successfully.


In [26]:
import shutil
from google.colab import files

zip_base_name = "/content/drive/MyDrive/fine_tuned_model"

# Zip the folder
shutil.make_archive(zip_base_name, 'zip', save_path)

# Define the zip file path
zip_file_path = zip_base_name + ".zip"

# Create a download link
files.download(zip_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>